# Sommaire

>[Sommaire](#scrollTo=OXXSgYy9BBec)

>[Imports et paramètres généraux](#scrollTo=zC7YhIrj59gy)

>[Création des modèles à entraîner](#scrollTo=gZFVwEW96EaG)

>[Fonctions d'affichages et d'enregistrements](#scrollTo=m_Wx43PK6pdE)

>[Entraînement](#scrollTo=5CAxdPTM6wkq)

>[Téléchargements](#scrollTo=4QJiqfKLBU_W)



# Imports et paramètres généraux

Imports des librairies principales et réglage des paramètres d'aléa

In [1]:
!pip install tqdm

# gestions relatives aux fichiers de manière générale
import os

# choix de tensorflow comme backend de Keras
os.environ["KERAS_BACKEND"] = "tensorflow"

# opérations matricielles
import numpy as np

from tqdm import tqdm

# affichages
import matplotlib.pyplot as plt

# les différentes parties utilisées de Keras
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

K.set_image_dim_ordering('th')

# on fixe le seed du random de numpy pour avoir des résultats reproductibles
np.random.seed(1000)

# On choisit la taille du vecteur aléatoire d'entrée du GAN
# 10 marche bien pour MNIST d'après la littérature
randomDim = 10

  Using cached https://files.pythonhosted.org/packages/c2/d4/bf1d99216672eef50392c009a8d0f282c4b643e4f764c962ff7b611ebce4/tqdm-4.23.1-py2.py3-none-any.whl


Using TensorFlow backend.


Chargement du dataset

In [0]:
# Chargement directement depuis Keras
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalisation des données
X_train = (X_train.astype(np.float32) - 127.5)/127.5

# Mise en forme des données d'entraînement
X_train = X_train.reshape(60000, 784)

# Création des modèles à entraîner

Configuration du Générateur

In [3]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Reshape

nb_filters = 32
kernel_size = (3,3)
input_shape = (randomDim,)
pool_size = (2,2)

generator = Sequential()

generator.add(Dense(28*28,input_shape=input_shape))

generator.add(Reshape((1,28,28),input_shape=input_shape))

generator.add(Convolution2D(nb_filters, kernel_size,
                             border_mode='valid',))
                             #input_shape=input_shape))
generator.add(Activation('relu'))
generator.add(MaxPooling2D(pool_size=pool_size))
generator.add(Dropout(0.5))

generator.add(Flatten())
generator.add(Dense(100))
generator.add(Activation('relu'))
generator.add(Dropout(0.5))

generator.add(Dense(784, activation='tanh'))

generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

generator.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid")`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 784)               8624      
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 28, 28)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 26, 26)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 26, 26)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 13, 13)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 13, 13)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
__________

Configuration du Discriminateur

In [4]:
# Architecture trouvée dans la littérature, marche bien pour MNIST
discriminator = Sequential()

discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))

discriminator.add(Dense(1, activation='sigmoid'))

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
__________

Configuration du GAN comme assemblage du Générateur et du Discriminateur

In [0]:
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Fonctions d'affichages et d'enregistrements

In [0]:
# Affichage de la fonction de coût
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('gan_loss_epoch_%d.png' % epoch)

# Affichage d'une mosaïque d'images générées par le Générateur
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [0]:
# sauvegarde des réseaux de neurones
def saveModels(epoch):
    generator.save('gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('gan_discriminator_epoch_%d.h5' % epoch)

# Entraînement

In [0]:
# Liste des Coûts du Discriminateur (un par epoch)
dLosses = []

# Liste des Coûts du Discriminateur (un par epoch)
gLosses = []

def train(epochs=1, batchSize=128):
  
    batchCount = X_train.shape[0] / batchSize
    
    # Affichage des paramètres de train
    print 'Epochs:', epochs
    print 'Batch size:', batchSize
    print 'Batches per epoch:', batchCount

    # ENTRAINEMENT
    
    # Pour chaque epoch
    for e in xrange(1, epochs+1):
      
        # Affichage du numéro de l'epoch
        print '-'*15, 'Epoch %d' % e, '-'*15
        
        # Pour chaque batch
        for _ in tqdm(xrange(batchCount)):
            # On génère des données random de la bonne taille pour générer <batchSize> fausses images 
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

            # On génère des fausses images à partir du bruit généré
            generatedImages = generator.predict(noise)
            X = np.concatenate([imageBatch, generatedImages])

            # J'ai pas trop compris ça, j'admets
            yDis = np.zeros(2*batchSize)
            yDis[:batchSize] = 0.9

            # Entrainement du Discriminateur
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            # Entrainement du Générateur, le Discriminateur étant immuable
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # Enregistrement des Coûts pour cette epoch
        dLosses.append(dloss)
        gLosses.append(gloss)

        # On enregistre les données relatives à l'entrainement régulièrement
        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e)

    # Affichage des coûts en fonction de l'epoch
    plotLoss(e)
    
iterations = 150
batchSize = 128

# C'est parti !
train(iterations, batchSize)

  0%|          | 0/468 [00:00<?, ?it/s]

Epochs: 150
Batch size: 128
Batches per epoch: 468
--------------- Epoch 1 ---------------


  1%|          | 3/468 [00:00<00:16, 28.16it/s]

--------------- Epoch 2 ---------------


  1%|          | 3/468 [00:00<00:17, 27.00it/s]

...

  1%|          | 3/468 [00:00<00:16, 28.66it/s]

--------------- Epoch 36 ---------------


  1%|          | 3/468 [00:00<00:16, 27.74it/s]

--------------- Epoch 37 ---------------


 53%|█████▎    | 249/468 [00:08<00:07, 28.23it/s]

# Téléchargements

In [1]:
from google.colab import files

for e in xrange(1, 300+1):
    if e == 1 or e % 20 == 0:
        files.download('gan_generated_image_epoch_%d.png' % e)
        files.download('gan_generator_epoch_%d.h5' % e)
        files.download('gan_discriminator_epoch_%d.h5' % e)

NameError: ignored